### Importaciones de paquetes necesarios

In [321]:
from turnosenfermeria import TurnosEnfermeria
import Alg_genetico_simple as alg_gen

# Algoritmo Genético Simple

In [322]:
tamano_individuo = 21
tamano_calendario = len (preferencias)
tamano_poblacion = 50
max_generaciones = 100
p_cruce = 0.9
p_mutacion = 0.1
algoritmo = "eaSimple"

toolbox = base.Toolbox()

In [323]:
# Comprobación del funcionamiento del algoritmo

poblacion_final, logbook = alg_gen.CreaccionCalendarioTurnos ("./data/datos_enfermeria.txt", tamano_individuo, tamano_calendario, tamano_poblacion, "selTournament", p_cruce, "cxOnePoint", p_mutacion, "mutFlipBit", turnos, max_generaciones)

C:\Users\paula\anaconda3\envs\practica_3_env\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'ClaseAjusteMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\paula\anaconda3\envs\practica_3_env\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'ClaseIndividuo' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


TypeError: 'str' object cannot be interpreted as an integer

## Mostramos el calendario por pantalla